In [1]:
import sys, os


In [2]:
from model import *
from load_data import *
from config import config
import torch
import torch.nn as nn
from torch.autograd import Variable
import torch.optim as optim
import argparse
import os
from torch.utils.data import DataLoader, Dataset, TensorDataset
import random
from tqdm import tqdm


In [3]:
test_data = load_raw('./data/test.csv')
test_data.head()

,id,text
0,id02310,"Still, as I urged our leaving Ireland with suc..."
1,id24541,"If a fire wanted fanning, it could readily be ..."
2,id00134,And when they had broken down the frail door t...
3,id27757,While I was thinking how I should possibly man...
4,id04081,I am not sure to what limit his knowledge may ...


In [4]:
tokenized_texts = test_data['text'].str.lower().apply(tokenize)

In [5]:
checkpoint = torch.load('best_checkpoint.pth.tar')

In [6]:
word_to_ix = checkpoint['word_to_ix']

In [7]:
net = Model(
    config.NUM_WORDS + 2,
    config.EMBEDDING_DIM,
    config.HIDDEN_SIZE,
    config.LINEAR_SIZE,
    config.NUM_CLASS,
    config.NLAYERS,
)

In [8]:
net.load_state_dict(checkpoint['state_dict'])

In [9]:
tokenized_ix = []
for tokenized_text in tokenized_texts:
    tokenized_ix.append(tokens_to_ix(word_to_ix, tokenized_text, fixed_length=64))
tokenized_ix = np.array(tokenized_ix)

In [10]:
test_dataloader = DataLoader(torch.LongTensor(tokenized_ix), batch_size = 48, shuffle=False)


In [11]:
len(word_to_ix)

10000

In [12]:
res = []
sm = nn.Softmax(dim=1)
for sentence in test_dataloader:
    var_x = Variable(sentence)
    
    pred = sm(net(var_x))
    res.append(pred)

In [13]:
result = res[0].data[0].numpy()

In [14]:
result

array([ 0.03122117,  0.00209413,  0.9666847 ], dtype=float32)

In [15]:
result = torch.cat(res)

In [16]:
result.shape

torch.Size([8392, 3])

In [17]:
result.data.numpy().shape

(8392, 3)

In [18]:
result_pd = pd.DataFrame(data=result.data.numpy(), index=test_data['id'], columns=['EAP', 'HPL', 'MWS'])

In [19]:
result_pd.head()

,EAP,HPL,MWS
id,,,
id02310,0.031221,0.002094,0.966685
id24541,0.904255,0.014239,0.081506
id00134,0.002616,0.993857,0.003527
id27757,0.897337,0.035359,0.067305
id04081,0.553061,0.181004,0.265935


In [20]:
result_pd.to_csv('submission.csv')